In [4]:
import os
import pandas as pd

from cube.api import Cube
from utils.text_data_transformation import transform_raw_data
from utils.dialog_manipulation import prepare_dialogs

In [13]:
# 0) Firstly, use data_transformation.py to get "subdialogs_{id}.csv" files


# 1) change to language of articles
lang = "ua"

# TODO: add -1 for all in dir
DIALOG_IDS = ["138918380", "470409323", "347963763", "850978724", "511986933"]
path_to_prepared_dialogs = os.path.join("data", "processed_dialog_files")

frames = []
df_idf = pd.DataFrame()
general_n_subdialogs = 0

for n_id, id in enumerate(DIALOG_IDS):
    path_to_dialog = os.path.join(path_to_prepared_dialogs, f"{id}", f"subdialogs_{id}.csv")

    data_for_df = []
    data = pd.read_csv(path_to_dialog)

    n_subdialog = 0
    dialog_dict, dialog_text = {}, ''
    for row in data.index:
        if n_subdialog != data["subdialog_id"][row]:
            n_subdialog += 1
            general_n_subdialogs += 1
            if n_subdialog != 1:
                dialog_dict["body"] = dialog_text
                data_for_df.append(dialog_dict)

            dialog_dict = {}
            dialog_text = ''
            dialog_dict["id"] = general_n_subdialogs
            dialog_dict["title"] = ""

        if not pd.isnull(data["message"][row]):
            dialog_text += " " + data["message"][row]

    df = pd.DataFrame(data_for_df)
    if n_id == 0:
        df_idf = df
    else:
        df_idf = pd.concat([df_idf, df])

# extend n_rows if you want to analyse keywords more
# df_idf = df_idf[:20]
df_idf

,id,title,body
0,1,,Наступного місяця бо зараз там якісь зміни в ...
1,2,,привіт діма написати функція просити десь виг...
2,3,,привіт діма книжка читати лаба річ почати пис...
3,4,,привіт діма дуже крутий пропозиція щодо стаду...
4,5,,привіт діма лаба зрозуміло дякувати дуже ради...
...,...,...,...
38,133,,facebook com якщій складно хай чий неактивний...
39,134,,привіт вови встигити постаратися завтра закид...
40,135,,пробувати зробити колонка стиля переставати п...
41,136,,привіт вови змозити субота colorlib com коман...


In [14]:
# # 1) write your number of rows to analyse. max == 695,
# # but to clean and analyse 30 rows is spending 6 minutes
# path_to_articles = os.path.join('data', 'processed_dialog_files', 'ukr_articles')
# data = pd.read_csv(path_to_articles, sep = ';', header = None)
# df_idf = pd.DataFrame(data[:20])
#
# df_idf.columns = ["id", "title", "en_title", "body", "date", "source_url",
#                   "article_url"]
#
# print("Schema:\n\n",df_idf.dtypes)
# print("Number of questions,columns=",df_idf.shape)
# df_idf

In [15]:

df_idf.dropna(subset = ["title"], inplace=True)
df_idf.dropna(subset = ["body"], inplace=True)

df_idf['text'] = df_idf['title'] + df_idf['body']

# if you have untransformed_data so uncomment it !!!!!!!!!!!!!
# if lang == "ua":
#     cube = Cube(verbose=True)
#     cube.load("uk")
# df_idf['text'] = df_idf['text'].apply(lambda x: transform_raw_data(x, lang, "words_frequency", cube))

#show the first 'text'
df_idf['text'][2]

2     привіт діма книжка читати лаба річ почати пис...
2     могти скинути приклад помилка valgrind мати в...
2     Еще нет Я занимался подачей документов на 6 о...
2     привіт вови хотітий уточнити варіант ставатис...
Name: text, dtype: object

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def get_stop_words():
    """load stop words """

    try:
        with open(os.path.join(os.getcwd(), "..", "dicts", "ukrainian_stopwords.txt"), "r", encoding="utf-8") as file:
            stop_words = str(file).strip().split()

    except FileNotFoundError:
        with open(os.path.join(os.getcwd(), "dicts", "ukrainian_stopwords.txt"), "r",
                  encoding="utf-8") as file:
            stop_words = str(file).strip().split()

    return stop_words

#load a set of stop words
stopwords=get_stop_words()

#get the text column
docs=df_idf['text'].tolist()

#create a vocabulary of words,
#ignore words that appear in 85% of documents,
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopwords)
word_count_vector=cv.fit_transform(docs)

d:\python\envs\venv_telegram_analysis\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['_io', 'analysis', 'dialogs', 'dicts', 'encoding', 'mode', 'my_work', 'name', 'programming', 'telegram', 'textiowrapper', 'txt', 'ukrainian_stopwords', 'utf'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [17]:
cv=CountVectorizer(max_df=0.85,stop_words=stopwords,max_features=10000, ngram_range=(2, 2))
word_count_vector=cv.fit_transform(docs)
word_count_vector.shape

(134, 10000)

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

# get test docs into a list
docs_test=df_idf['text'].tolist()
docs_title=df_idf['title'].tolist()
docs_body=df_idf['body'].tolist()

In [19]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""

    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]

        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]

    return results

# you only needs to do this once
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs_test[0]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)

# now print the results
print("\n=====Title=====")
print(docs_title[0])
print("\n=====Body=====")
print(docs_body[0])
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Title=====


=====Body=====
 Наступного місяця бо зараз там якісь зміни в управлінні і не проводять іспити  Добре ти правий так і зробимо Привіт  Коли у тебе до речі по водінню іспит  Ти п яте ще не робив  Щоб пошвидше Просто потрібно спочатку скинути йому щоб він переглянув а потім уже будемо додаткові робити Привіт Діма ти може бенчмарк поки можеш не добавляти або у окрему гілку якусь написати активно займатися лаба річ частково зробити твій функція масив гіт закидати ятий завдання зробити привіт функція зчитування частково привіт діма завдання зробити треба робити сподіватися понеділок мати готово завтра допрацьовувати пушнути окий окойа спокійно дивитися тепер дуже добре розбиратися працювати дати безкоштовно радий пропасти страх код аха форма заповнити тип студент страшно написати viva64 com більший кількість помилка вказувати скачати pvc валгрінор пройтися зробити справа перший лаба треба зробити тобто код інший модуль запуститися змогти використати порівняння взагалі викор

In [20]:
# put the common code into several methods
def get_keywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs_test[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)

    return keywords

def print_results(idx,keywords):
    # now print the results
    print("\n=====Title=====")
    print(docs_title[idx])
    print("\n=====Body=====")
    print(docs_body[idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])

In [21]:
#generate tf-idf for all documents in your list. docs_test has 500 documents
tf_idf_vector=tfidf_transformer.transform(cv.transform(docs_test))

results=[]
for i in range(tf_idf_vector.shape[0]):

    # get vector for a single document
    curr_vector=tf_idf_vector[i]

    #sort the tf-idf vector by descending order of scores
    sorted_items=sort_coo(curr_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)


    results.append(keywords)

df=pd.DataFrame(zip(docs,results),columns=['doc','keywords'])
for result in results:
    result2 = {k: v for k, v in sorted(result.items(), key=lambda item: item[1], reverse=True)}
    print(result2)

df.to_csv(os.path.join("data", "processed_dialog_files", "subdialog_keywords.csv"))
print("result df")
df

{'index str': 0.111, 'мати декілька': 0.103, 'const str': 0.097, 'github com': 0.091, 'треба вирішити': 0.074, 'статичний типізація': 0.074, 'порівнювати код': 0.074, 'може написати': 0.074, 'ліба пайтон': 0.074, 'завдання функція': 0.074}
{'файл str': 0.365, 'str functions': 0.317, 'функція просити': 0.183, 'модуль розбити': 0.183, 'github com': 0.112, 'інтегрувати старий': 0.091, 'який бренчий': 0.091, 'чогось потрібен': 0.091, 'хотіти справа': 0.091, 'хотіти свій': 0.091}
{'щось навчання': 0.201, 'займатися щось': 0.201, 'їда батько': 0.1, 'якось data': 0.1, 'якийсь курся': 0.1, 'язаний робити': 0.1, 'що навчитися': 0.1, 'шукати кіось': 0.1, 'швидше спорт': 0.1, 'читати лаба': 0.1}
{'щодо стадування': 0.361, 'тип лаба': 0.361, 'стадування тип': 0.361, 'пік відходити': 0.361, 'відходити лекція': 0.361, 'пропозиція щодо': 0.333, 'діма дуже': 0.333, 'дуже крутий': 0.297, 'привіт діма': 0.201}
{'порівняння могти': 0.2, 'могти блока': 0.2, 'блока почати': 0.2, 'функція пошук': 0.174, 'по

,doc,keywords
0,Наступного місяця бо зараз там якісь зміни в ...,"{'index str': 0.111, 'мати декілька': 0.103, '..."
1,привіт діма написати функція просити десь виг...,"{'файл str': 0.365, 'str functions': 0.317, 'ф..."
2,привіт діма книжка читати лаба річ почати пис...,"{'щось навчання': 0.201, 'займатися щось': 0.2..."
3,привіт діма дуже крутий пропозиція щодо стаду...,"{'щодо стадування': 0.361, 'тип лаба': 0.361, ..."
4,привіт діма лаба зрозуміло дякувати дуже ради...,"{'порівняння могти': 0.2, 'могти блока': 0.2, ..."
...,...,...
129,facebook com якщій складно хай чий неактивний...,"{'profession json': 0.265, 'open path': 0.265,..."
130,привіт вови встигити постаратися завтра закид...,"{'язаний колір': 0.226, 'хороший робот': 0.226..."
131,пробувати зробити колонка стиля переставати п...,"{'являтися сторінка': 0.158, 'чудово практично..."
132,привіт вови змозити субота colorlib com коман...,"{'html сторінка': 0.253, 'який хочий': 0.127, ..."
